In [ ]:
!pip install split-folders

In [ ]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')



Mounted at /content/drive/


In [ ]:
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/archive(1).zip", 'r')
zip_ref.extractall("/content")
zip_ref.close()

In [ ]:
import splitfolders
from keras.applications.vgg19 import VGG19
from keras import models
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np

# A. Split the weather dataset into three train, validation, and test folders with a ratio of 0.8, 0.1, and 0.1 using the splitfolders library.

input_folder = '/content/dataset'
output_folder = '/content/output_folder'

splitfolders.ratio(input_folder, output_folder, seed=42, ratio=(0.8, 0.1, 0.1))

# B. Use the VGG19 network which trained on the ImageNet dataset. The VGG19 has the following architecture:

base_model = VGG19(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# C. Resize the images to 150 x 150.

train_datagen = ImageDataGenerator(rescale=1./255,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = output_folder + '/train'
val_dir = output_folder + '/val'
test_dir = output_folder + '/test'

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(val_dir,
                                                        target_size=(150, 150),
                                                        batch_size=32,
                                                        class_mode='categorical')


# D. Remove the network's top layers containing the classification layers.

model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(11, activation='softmax'))

base_model.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# E. Use the VGG19 model for feature extraction without data augmentation.

history = model.fit(train_generator,
                    epochs=50,
                    batch_size=32,
                    validation_data=validation_generator)

# F. Add dense layers for the classification part.

base_model.trainable = True

set_trainable = False
for layer in base_model.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

history_fine_tuned = model.fit(train_generator,
                               epochs=50,
                               batch_size=32,
                               validation_data=validation_generator)

# G. In this part, extract features using data augmentation, which means extending the VGG19 model and running it end to end on the inputs.

train_datagen_aug = ImageDataGenerator(rescale=1./255,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True)

test_datagen_aug = ImageDataGenerator(rescale=1./255)

train_generator_aug = train_datagen_aug.flow_from_directory(train_dir,
                                                            target_size=(150, 150),
                                                            batch_size=32,
                                                            class_mode='categorical')

validation_generator_aug = test_datagen_aug.flow_from_directory(val_dir,
                                                                target_size=(150, 150),
                                                                batch_size=32,
                                                                class_mode='categorical')

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

history_aug = model.fit(train_generator_aug,
                        epochs=50,
                        batch_size=32,
                        validation_data=validation_generator_aug)

# Plotting the accuracy and loss for feature extraction without data augmentation

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy without data augmentation')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss without data augmentation')
plt.legend()

plt.figure()

# Plotting the accuracy and loss for fine-tuned VGG19 model

acc = history_fine_tuned.history['accuracy']
val_acc = history_fine_tuned.history['val_accuracy']
loss = history_fine_tuned.history['loss']
val_loss = history_fine_tuned.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy with fine-tuned VGG19 model')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss with fine-tuned VGG19 model')
plt.legend()

plt.figure()

# Plotting the accuracy and loss for feature extraction with data augmentation

acc = history_aug.history['accuracy']
val_acc = history_aug.history['val_accuracy']
loss = history_aug.history['loss']
val_loss = history_aug.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy with feature extraction and data augmentation')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss with feature extraction and data augmentation')
plt.legend()

plt.show()

In [ ]:
import splitfolders
from keras.applications.vgg19 import VGG19
from keras import models
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np

# A. Split the weather dataset into three train, validation, and test folders with a ratio of 0.8, 0.1, and 0.1 using the splitfolders library.
input_folder = '/content/dataset'
output_folder = '/content/output_folder'

splitfolders.ratio(input_folder, output_folder, seed=42, ratio=(0.8, 0.1, 0.1))

# B. Use the VGG19 network which trained on the ImageNet dataset.
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# C. Resize the images to 150 x 150 and apply data augmentation to the training set.
train_datagen = ImageDataGenerator(rescale=1./255,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = output_folder + '/train'
val_dir = output_folder + '/val'
test_dir = output_folder + '/test'

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(val_dir,
                                                        target_size=(150, 150),
                                                        batch_size=32,
                                                        class_mode='categorical')

# D. Remove the network's top layers containing the classification layers and add dense layers for the classification part, including batch normalization.
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(11, activation='softmax'))

base_model.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# E. Fine-tune the model by making the last convolutional block trainable.
for layer in base_model.layers:
    if layer.name.startswith('block5'):
        layer.trainable = True
    else:
        layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

history_fine_tuned = model.fit(train_generator,
                               epochs=50,
                               batch_size=32,
                               validation_data=validation_generator)

# F. Plot the accuracy and loss for training and validation data.
acc = history_fine_tuned.history['accuracy']
val_acc = history_fine_tuned.history['val_accuracy']
loss = history_fine_tuned.history['loss']
val_loss = history_fine_tuned.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy (fine-tuned last block)')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss (fine-tuned last block)')
plt.legend()

plt.show()

In [ ]:
import splitfolders
from keras.applications.vgg19 import VGG19
from keras import models
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np

# A. Split the weather dataset into three train, validation, and test folders with a ratio of 0.8, 0.1, and 0.1 using the splitfolders library.
input_folder = '/content/dataset'
output_folder = '/content/output_folder'

splitfolders.ratio(input_folder, output_folder, seed=42, ratio=(0.8, 0.1, 0.1))

# B. Use the VGG19 network which trained on the ImageNet dataset.
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# C. Resize the images to 150 x 150 and apply data augmentation to the training set.
train_datagen = ImageDataGenerator(rescale=1./255,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                  zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = output_folder + '/train'
val_dir = output_folder + '/val'
test_dir = output_folder + '/test'

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=32,
                                                    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(val_dir,
                                                        target_size=(150, 150),
                                                        batch_size=32,
                                                        class_mode='categorical')

# D. Remove the network's top layers containing the classification layers and add dense layers for the classification part, including batch normalization.
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(11, activation='softmax'))

base_model.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# E. Fine-tune the model by making the last two convolutional blocks trainable.
for layer in base_model.layers:
    if layer.name.startswith('block4') or layer.name.startswith('block5'):
        layer.trainable = True
    else:
        layer.trainable = False

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

history_fine_tuned = model.fit(train_generator,
                               epochs=50,
                               batch_size=32,
                               validation_data=validation_generator)

# F. Plot the accuracy and loss for training and validation data.
acc = history_fine_tuned.history['accuracy']
val_acc = history_fine_tuned.history['val_accuracy']
loss = history_fine_tuned.history['loss']
val_loss = history_fine_tuned.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy (fine-tuned last two blocks)')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss (fine-tuned last two blocks)')
plt.legend()

plt.show()